In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from util import DenseTransformer, MyTokenizer, MeanEmbeddingVectorizer, get_word2vec, MySentences
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, cross_validate
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from tqdm import tqdm
from gensim.models import word2vec
import nltk

In [3]:
df = pd.read_csv('./data/dataset_with_content.csv', index_col=0, parse_dates=True)

In [116]:
X = df.t +'\n'+ df.content
y = df['pct_change'] > 0

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
X_train = X_train.astype(str)
X_test = X_test.astype(str)

In [79]:
tfidf_tf = TfidfVectorizer(stop_words='english', ngram_range=(2,2))
x = tfidf_tf.fit_transform(X_train.astype(str)).toarray()

In [80]:
pd.DataFrame([tfidf_tf.vocabulary_]).T.sort_values(0).head()

,0
00 00,0
00 11,1
00 30,2
00 agenda,3
00 cdt,4


In [81]:
len(tfidf_tf.get_feature_names())

105981

In [57]:
idx = pd.DataFrame(x, columns=tfidf_tf.get_feature_names())['00'] > 0

In [91]:
samples = X_train.loc[X_train.index[idx]][:5]
samples[0]

"Federal Reserve offers seven-day term deposits through its Term Deposit Facility\n\n\n       On June 23, 2014, the Federal Reserve will conduct a fixed-rate offering of term deposits through its Term Deposit Facility (TDF). The Federal Reserve will offer seven-day term deposits with an interest rate of 0.28000 percent and a maximum tender amount of $10,000,000,000. As noted in the Federal Reserve Board's May 9, 2014, release, this operation is part of the continuing program of operational testing of its policy tools designed to provide eligible institutions with an opportunity to gain familiarity with term deposit operations. Additional information regarding the operation is listed below; the operation will be conducted as specified in this announcement, Regulation D, and the terms and conditions of the Term Deposit Facility (http://www.frbservices.org/centralbank/term_deposit_facility.html).\n    \n\nDescription of Operation and Tender Parameters\n\n\n\n\n          TDF Operation ID:\

In [102]:
vec = TfidfVectorizer(analyzer='word', token_pattern='[a-z]+', stop_words='english', ngram_range=(2,2))
vec.fit(samples)
len(vec.get_feature_names())

449

In [107]:
pd.DataFrame(vec.get_feature_names()).sample(10)

,0
134,et notification
335,rate shown
363,rounding convention
15,approval applications
129,error rounding
282,operation id
199,individual institutions
297,parameters listed
56,billion day
434,website board


In [109]:
vec.fit(X_train.astype(str))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='[a-z]+', tokenizer=None, use_idf=True,
        vocabulary=None)

In [165]:
pipeline=make_pipeline(MeanEmbeddingVectorizer(w2v_model), SVC())
cross_val_score(pipeline, X_train, y_train, cv=5, scoring='roc_auc')

/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packa

array([0.5237069 , 0.54411024, 0.52568116, 0.4463016 , 0.53891364])

In [161]:
pipeline=make_pipeline(MeanEmbeddingVectorizer(w2v_model), LogisticRegression(solver='lbfgs'))
cross_val_score(pipeline, X_train, y_train, cv=5, scoring='roc_auc')

/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the nu

array([0.54129436, 0.53648062, 0.51306475, 0.48441826, 0.50744146])

In [4]:
df['text'] = df['t'] + '\n' + df['content']
df['text'] = df['text'].astype(str)
X_train, X_test, y_train, y_test =  train_test_split(df[df.columns.difference(['pct_change'])], (df['pct_change'] >0).astype(int))

In [6]:
pipeline=make_pipeline(OneHotEncoder(), LogisticRegression(solver='lbfgs'))
cross_val_score(pipeline, X_train.pt.values.reshape(-1,1), y_train, cv=5, scoring='roc_auc')

array([0.51959525, 0.49439183, 0.47540429, 0.44234716, 0.48106132])

In [7]:
# class FeatureInteractionTransformer(TransformerMixin):
#     def __init__(self, main_col):
#         self.main_col = main_col 

#     def fit(self, X, y=None, **fit_params):
#         return self

#     def transform(self, X, y=None, **fit_params):
#         return X.todense()

# pipeline=make_pipeline(MeanEmbeddingVectorizer(w2v_model), LogisticRegression(solver='lbfgs'))
# cross_val_score(pipeline, X_train, y_train, cv=5, scoring='roc_auc')

NameError: name 'TransformerMixin' is not defined

In [5]:
w2vec = get_word2vec(
    MySentences(
         X_train['text'].values
    ),
    'w2vmodel'
)

Found w2vmodel


In [7]:
pipeline=make_pipeline(MeanEmbeddingVectorizer(w2vec), SVC())
cross_val_score(pipeline, X_train['text'].values, y_train, cv=5, scoring='roc_auc')

/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packa

array([0.53757913, 0.53536662, 0.54532238, 0.52922222, 0.50661111])

In [23]:
# well that sucks

In [22]:
col_tf = ColumnTransformer([
        ('w2v', MeanEmbeddingVectorizer(w2vec), ['text']),
        ('onehot', OneHotEncoder(), ['pt'])
    ],
    remainder='drop'
)

pipeline=make_pipeline(col_tf, SVC())
cross_val_score(pipeline, X_train, y_train, cv=5, scoring='roc_auc')

/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 2022, expected 1.

In [29]:
col_tf = ColumnTransformer([
        ('w2v', MeanEmbeddingVectorizer(w2vec), ['text']),
        ('onehot', OneHotEncoder(), ['pt'])
    ],
    remainder='drop'
)

col_tf.fit_transform(X_train)

ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 2529, expected 1.

In [40]:
r = MeanEmbeddingVectorizer(w2vec).fit_transform(X_train.text)

In [56]:
h = OneHotEncoder().fit_transform(X_train.pt.values.reshape(-1,1)).toarray()

In [57]:
r.shape

(2529, 100)

In [58]:
h.shape

(2529, 5)

In [59]:
_X = np.hstack((r,h))

In [60]:
svm_clf = SVC()
cross_val_score(svm_clf, _X, y_train, cv=5, scoring='roc_auc')

/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packa

array([0.54749945, 0.52003793, 0.53732617, 0.52338095, 0.50915079])

In [84]:
X_train.d = pd.to_datetime(X_train.d)
X_train['hour'] = X_train.d.dt.hour
X_train['dayofweek'] = X_train.d.dt.dayofweek
X_train['month'] = X_train.d.dt.month
X_train['year'] = X_train.d.dt.year
dates = X_train[['hour', 'dayofweek', 'month', 'year']]

In [85]:
_X = np.hstack((_X, dates.values))

In [86]:
_X.shape

(2529, 109)

In [87]:
cross_val_score(svm_clf, _X, y_train, cv=5, scoring='roc_auc')

/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/harinsa/miniconda3/lib/python3.7/site-packa

array([0.57114288, 0.4835019 , 0.52029077, 0.57525397, 0.5314127 ])

In [88]:
SVC()

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [92]:
for gamma in [0.01, 0.1, 1., 5]:
    scores= cross_val_score(SVC(gamma=gamma), _X, y_train, cv=5, scoring='roc_auc')
    print(gamma, scores)

0.01 [0.57085945 0.48174779 0.52021176 0.5762381  0.53212698]
0.1 [0.57155229 0.50323957 0.5147914  0.58239683 0.58471429]
1.0 [0.61170598 0.56267383 0.59519595 0.61463492 0.57826984]
5 [0.6037776  0.54871997 0.60079014 0.60450794 0.58561905]


In [96]:
for i in [100, 200, 500, 1000]:
    scores = cross_val_score(XGBClassifier(n_estimators=i, n_jobs=-1), _X, y_train, cv=5, scoring='roc_auc')
    print(i, scores)

100 [0.53045382 0.52179204 0.5585493  0.51546032 0.49327778]
200 [0.54326363 0.51089602 0.56189949 0.52593651 0.49970635]
500 [0.55156993 0.50775917 0.5715866  0.54255556 0.50525397]
1000 [0.55076686 0.50891277 0.56959545 0.55034921 0.50639683]


In [100]:
for c in [1, 10, 100, 1000]:
    scores= cross_val_score(LogisticRegression(C=c, solver='lbfgs', max_iter=1000), _X, y_train, cv=5, scoring='roc_auc')
    print(c, scores)

1 [0.52387176 0.51961125 0.52757585 0.50195238 0.49790476]
10 [0.51121154 0.50775917 0.53358091 0.48795238 0.49612698]
100 [0.52558813 0.52051201 0.51825221 0.49149206 0.50993651]
1000 [0.52045476 0.51273704 0.53245891 0.5218254  0.51477778]


In [104]:
svm = SVC(gamma=1)
cross_val_score(svm, dates.values, y_train, cv=5, scoring='roc_auc')

array([0.67144049, 0.58626738, 0.6187579 , 0.63388889, 0.64015079])

In [106]:
from sklearn.ensemble.partial_dependence import plot_partial_dependence

In [111]:
plot_partial_dependence(svm, dates.values, range(4), n_cols=4, feature_names=dates.columns)

ValueError: gbrt has to be an instance of BaseGradientBoosting